### Notebook to perform Zernike polynominal analysis of SKA1-LOW and LOFAR calibration

In [ ]:
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 8.0)
#pylab.rcParams['image.cmap'] = 'rainbow'

from telutil import *

import subprocess
print "Working from Git repository %s" % subprocess.check_output(["git",  "describe", "--long", "--all"])

def timestamp():
    from datetime import datetime
    print "%s %s" % (datetime.now().date(), datetime.now().time())    

def trials(ntrials,nsources,config,name,rmin=0,rmax=200.0,tiono=10,tsky=10000*3600.0,snrthreshold=5.0):
    tp=TelPiercings()

    timestamp()

    for trial in range(ntrials):
        ts=TelSources()
        ts.construct(nsources=nsources, radius=wavelength/stationdiameter)
        tp.construct(ts,config, weight=weight, hiono=hiono)
        if trial==0:
            tp.plot(rmax=HWZ)

        s=numpy.sqrt(tp.assess(nnoll=nnoll, rmax=HWZ, doplot=(trial==0)))
        print "Trial %d, max(s)=%.1f" % (trial, numpy.max(s))

        if trial==0:
            Save=s/float(ntrials)
        else:
            Save=Save+s/float(ntrials)
    J=len(Save[Save>snrthreshold])
    if J<1:
        J=1
    DR=TelIono().dr(J,tsky=tsky,B=2*HWZ)
    itsky=TelIono().tsky(J,DR=1e5,B=2*HWZ)
    stdphase=numpy.sqrt(TelIono().varphase(J,B=2*HWZ))

    return {'dr':DR, 's':Save, 'J':J, 'tsky':itsky, 'stdphase':stdphase}

def printstats(stats):
    print "Config, Nsources, peak, J, stdphase, DR, sky, MST" 

    for config in configs:
        for nsources in lsources:
            print "%s, %d, %.1f, %d, %.2f, %.1f, %.1f, %.1f" \
            % (config, nsources, stats[config][nsources]['s'][0], \
               stats[config][nsources]['J'], \
               stats[config][nsources]['stdphase'], \
            10.0*numpy.log10(stats[config][nsources]['dr']), \
               stats[config][nsources]['tsky']/(24.0*3600.0*365.0), 
               mst[config])

random.seed(781490893)

nstations=512
nnoll=1000

wavelength=6.0
stationdiameter=35.0
rcore=0.5
rmin=0.0
hiono=300.0

HWZ=hiono*wavelength/stationdiameter # Half width to Zero
FOV=(0.5*wavelength/stationdiameter)**2

rhalo=HWZ
rmax=HWZ
print "FOV (HWZ) at ionosphere=%.1f km" % HWZ

# Frequency
freq=3.0e8/wavelength
print "Observing frequency = %.2f MHz" % (freq/1e6)

tiono=10.0*numpy.power(wavelength/3.0,-5.0/6.0)
print "Ionospheric coherence time = %.1f (s)" % (tiono)

# Calculate weight of solution for each pierce point
imgnoise=sources().tnoise(time=tiono, freq=freq)*(float(nstations)/512)*(35.0/stationdiameter)**2
print "Image noise in %.1f s = %.3f Jy" % (tiono, imgnoise)

visnoise=sources().tnoise(time=tiono, freq=freq)*float(512)*(35.0/stationdiameter)**2
print "Visibility noise in %.1f s = %1.1f Jy" % (tiono, visnoise)

gainnoise=visnoise/numpy.sqrt(float(nstations))

imgthreshold=5.0*gainnoise
# Weight is the gain for a 1.0 Jy source"
weight=1.0/gainnoise**2

print "Weight for 1 Jy source = %.1f (1/Jy^2)" % (weight)

# Number of sources in the beam abouve 5 sigma in tiono
nsources=FOV*sources().numbers(imgthreshold)
print "%.1f sources above pierce threshold (%.4f Jy/beam)" % (nsources, imgthreshold)

ntrials=10
lsources=[1,2]

### Define all configs and plot

In [ ]:

stats={}
tel={}
mst={}
configs = ['LOWBD2', 'LOWBD2-CORE', 'LOWBD2-RASTERHALO','LOWBD2-RASTERHALO%.0fMHz' % (freq*1e-6)]

for config in configs:

# Define all config variants
    tel[config]=TelArray()
    if config=='LOWBD2':
        tel[config].readKML(kmlfile='V4Drsst512red_2.kml', weight=weight)
        tel[config].saveCSV('LOWBD2.csv')
    elif config =='LOWBD2-CORE':
        tel[config].readCSV('LOWBD2-CORE', l1def='LOWBD2-CORE.csv', recenter=True, weight=weight)
    elif config =='LOWBD2-HALO':
        tel[config].readCSV('LOWBD2-HALO', l1def='LOWBD2-HALO.csv', recenter=True, weight=weight)
    elif config == 'LOWBD2-RASTERHALO':
        tel['RASTERHALO']=TelArray()
        tel['RASTERHALO'].rasterBoolardy(nhalo=346, name='LOWRASTERHALO', nstations=346, scale=1.0, \
                           rhalo=40.0, rcore=0.0, weight=weight)
        tel[config]=TelArray().add(tel['RASTERHALO'],tel['LOWBD2-CORE'],name=config)
    elif config == 'LOWBD2-RASTERHALO%.0fMHz' % (freq*1e-6):
        tel['RASTERHALO%.0fMHz' % (freq*1e-6)]=TelArray()
        tel['RASTERHALO%.0fMHz' % (freq*1e-6)].rasterBoolardy(nhalo=346, 
                                                              name='LOWRASTERHALO%.0fMHz' % (freq*1e-6), 
                                                              nstations=346, scale=0.9,
                                                              rhalo=HWZ, rcore=0.0, weight=weight)
        tel[config]=TelArray().add(tel['RASTERHALO%.0fMHz' % (freq*1e-6)],tel['LOWBD2-CORE'],name=config)
    elif config =='LOFAR':
        tel[config].readLOFAR(weight=weight)
# Plot and save the figure        
    tel[config].plot(rmax=HWZ)
    plt.show()
    plt.savefig('%s.pdf'%config)

for config in configs:
    mst[config]=tel[config].mst()

### Now assess all configs for varying numbers of sources

In [ ]:

stats={}

for nsource in lsources:

    for config in configs:
        print "Processing %s %d sources" % (config, nsource)
        timestamp()
        if nsource==lsources[0]:
            stats[config]={}
        stats[config][nsource]=trials(ntrials, nsource, tel[config], config)      
        
timestamp()

### Now plot results for all the configs

In [ ]:
plt.clf()

for nsource in lsources:
    ymax=0.1
    for config in stats.keys():
        y=stats[config][nsource]['s']
        if numpy.max(y)>ymax:
            ymax=numpy.max(y)
        x=numpy.arange(nnoll)
        plt.semilogy(x, y, label=config)
    plt.legend(loc="upper right")
    plt.xlim([1,1000])
    plt.ylim([0.1,ymax])
    plt.xlabel('Singular vector index')
    plt.ylabel('Singular value')
    plt.title('Singular value spectra for different configurations: %d source' % nsource)
    plt.show()

    plt.savefig('SingularValues_sources%d.pdf' % nsource)

In [ ]:
plt.clf()

for config in stats.keys():
    i=0
    x=numpy.zeros(len(lsources))
    y=numpy.zeros(len(lsources))
    for nsource in lsources:
        x[i]=nsource
        y[i]=stats[config][nsource]['tsky']/(365.0*24.0*3600.0)
        i=i+1
    plt.semilogy(x, y, label=config)
plt.legend(loc="upper right")
plt.xlabel('Number of sources')
plt.xlim([0,35])
plt.ylabel('Time on-sky to reach 50dB (years)')
plt.title('Time on-sky to reach 50dB vs number of sources')
plt.show()
plt.savefig('Timeonsky.pdf')
    

In [ ]:
plt.clf()

config = 'LOWBD2'

for nsource in lsources:
    ymax=0.1
    y=stats[config][nsource]['s']
    if numpy.max(y)>ymax:
        ymax=numpy.max(y)
    x=numpy.arange(nnoll)
    plt.semilogy(x, y, label='Number of sources %d' %nsource)

plt.legend(loc="upper right")
plt.xlim([1,500])
plt.ylim([0.1,ymax])
plt.xlabel('Singular vector index')
plt.ylabel('Singular value')
plt.title('Singular value spectra for different numbers of sources %s' % (config))
plt.show()
plt.savefig('%s_Sources.pdf' % config)

In [ ]:
printstats(stats)